In [1]:
'''
Notebook for FIG S11I - effect of native gene mutaitions increasing resistance to chloramphenicol
'''
# By Kirill Sechkar

# PACKAGE IMPORTS ------------------------------------------------------------------------------------------------------
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
import pickle
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, palettes as bkpalettes, transform as bktransform
from math import pi
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
from contourpy import contour_generator as cgen
import time

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# set up bokeh
bkio.reset_output()
bkio.output_notebook() 

# OWN CODE IMPORTS -----------------------------------------------------------------------------------------------------
import synthetic_circuits as circuits
from cell_model import *
from get_steady_state import *
from Fig2.design_guidance_tools import *

/tmp/ipykernel_1699/2897787407.py:25: DeprecationWarning: jax.lib.xla_bridge.get_backend is deprecated; use jax.extend.backend.get_backend.
  print(xla_bridge.get_backend().platform)
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


cpu


Loading BokehJS ...

In [2]:
 # INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, circuit_eff_m_het_div_k_het,  \
    par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.punisher_b_initialise,
    circuits.punisher_b_ode,
    circuits.punisher_b_F_calc,
    circuits.punisher_sep_b_eff_m_het_div_k_het,
    par, init_conds)  # load the circuit

In [3]:
# SPECIFY THE CIRCUIT'S DEFAULT PARAMETERS

# BURDENSOME SYNTHETIC GENE
par['c_b'] = 1  # gene concentration (nM)
par['a_b'] = 1e5    # promoter strength (unitless)

# PUNISHER
# switch gene conc
par['c_switch'] = 10.0  # gene concentration (nM)
par['a_switch'] = 400.0  # promoter strength (unitless)
par['d_switch']=0.01836 # rate of protein degradation per protease molecule (1/nM/h)
# integrase - expressed from the switch gene's operon, not its own gene => c_int, a_int irrelevant
par['k+_int'] = par['k+_switch']/80.0  # RBS weaker than for the switch gene
par['d_int'] = 0.0  # rate of integrase degradation per protease molecule (1/nM/h)
# CAT (antibiotic resistance) gene
init_conds['cat_pb'] = 10.0  # gene concentration (nM) - INITIAL CONDITION< NOT PARAMETER as it can be cut out by the integrase
par['a_cat'] = 500.0  # promoter strength (unitless)
# synthetic protease gene
par['c_prot'] = 10.0  # gene concentration (nM)
par['a_prot'] = 25.0  # promoter strength (unitless)
init_conds['p_prot'] = 1500.0 # if zero at start, the punisher's triggered prematurely

# punisher's transcription regulation function
par['K_switch'] = 300.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
par['eta_switch'] = 2 # Hill coefficient for the self-activating switch gene promoter (unitless)
par['baseline_switch'] = 0.025  # Baseline value of the switch gene's transcription activation function
par['p_switch_ac_frac'] = 0.85  # active fraction of protein (i.e. share of molecules bound by the inducer)

# CULTURE MEDIUM
init_conds['s'] = 0.5   # nutrient quality (unitless)
par['h_ext'] = 10.5 * (10.0 ** 3)   # nutrient concentration in the medium (nM)

In [4]:
# DETERMINISTIC SIMULATION PARAMETERS (FOR FINDING GROWTH RATES)
savetimestep = 1  # save time step
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver
# simulation time frames
tf = (0,50) # time frame for simulation before burdensome gene loss

In [6]:
# FIND STEADY-STATE GROWTH RATES FOR DIFFERENT CHLOR

# define the range of chloramphenicol diffusion coefficients for flux across the cell membrane
eff_h_range = np.concatenate((np.array([0]),np.logspace(0,6,13)))

l_ss=np.zeros_like(eff_h_range)    # steady-state growth rates WITH CAT
l_ss_nocat_nob=np.zeros_like(eff_h_range)    # steady-state growth rates WITHOUT CAT AND BURDEN
for i in range(0,len(eff_h_range)):
    cur_par = par.copy() # current parameter dictionary
    cur_par['eff_h'] = eff_h_range[i] # set the burdensome gene promoter strength
    cur_par['k_sxf']=0.0    # not considering punisher action explicitly
    cur_inits = init_conds.copy() # current initial conditions
    
    # get the steady-state growth rate WTIH CAT
    sol = ode_sim(cur_par,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(cur_inits, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(cur_par, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
    ts = np.array(sol.ts)
    xs = np.array(sol.ys)
    _, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, cur_par, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                           circuit_eff_m_het_div_k_het)
    l_ss[i]=np.float64(ls[-1])
    
    # get the steady-state growth rate WITHOUT CAT AND BURDEN
    cur_inits['cat_pb'] = 0.0 # set the CAT gene's concentration to zero
    cur_par['func_b'] = 0.0 # set the burdensome gene's functionality to zero
    sol_nocat_nob = ode_sim(cur_par,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(cur_inits, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(cur_par, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
    ts_nocat_nob = np.array(sol_nocat_nob.ts)
    xs_nocat_nob = np.array(sol_nocat_nob.ys)
    _, ls_nocat_nob, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts_nocat_nob, xs_nocat_nob, cur_par, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                           circuit_eff_m_het_div_k_het)
    l_ss_nocat_nob[i]=np.float64(ls_nocat_nob[-1])
                  


In [8]:
# PLOT

# set the plot ranges
l_range = (0.3, 1.6)

# set up the figure
k0kl_fig = bkplot.figure(
    frame_width=250,
    frame_height=150,
    x_axis_label="κeff (chloramphenicol res. fold-improvement)",
    y_axis_label="λ (steady-state\ngrowth rate), 1/h",
    x_range=(min(eff_h_range[1:]), max(eff_h_range[1:])),
    y_range=l_range,
    x_axis_type='log',
    tools="box_zoom,pan,hover,reset,save"
)
k0kl_fig.xaxis.axis_label_text_font_size = '9pt'
k0kl_fig.yaxis.axis_label_text_font_size = '9pt'
k0kl_fig.output_backend = 'svg'
# plot the growth rates WITH CAT
k0kl_fig.line(eff_h_range[1:], l_ss[1:],
               line_width=2, color='blue', legend_label='with CAT & burden')
k0kl_fig.scatter(eff_h_range[1:], l_ss[1:],
                  marker='circle', line_width=2,
                  color='blue', legend_label='with CAT & burden')
# plot the growth rates WITHOUT CAT AND BURDEN
k0kl_fig.line(eff_h_range[1:], l_ss_nocat_nob[1:],
               line_width=2, color='red', legend_label='w/out CAT & burden')
k0kl_fig.scatter(eff_h_range[1:], l_ss_nocat_nob[1:],
                  marker='circle', line_width=2,
                  color='red', legend_label='w/out CAT & burden')
# mark the original growth rate
k0kl_fig.add_layout(bkmodels.Span(location=l_ss[0], dimension='width',
                                   line_color='violet', line_dash='dashed', line_width=3))


# set up the legend
k0kl_fig.legend.location = 'left'
k0kl_fig.legend.glyph_width = 10
k0kl_fig.legend.label_text_font_size = '8pt'
k0kl_fig.legend.click_policy = 'hide'
k0kl_fig.legend.title_text_font_size = '8pt'
k0kl_fig.legend.background_fill_alpha = 1
k0kl_fig.legend.border_line_alpha = 1
k0kl_fig.legend.margin = 5
k0kl_fig.legend.spacing = 2
k0kl_fig.legend.padding = 2

# show the plot
bkplot.show(k0kl_fig)